In [1]:
import duckdb
import pandas

In [ ]:
import faker
import uuid
import random, time, datetime
import time

: 

In [3]:
fake = faker.Faker()

In [ ]:

device = ["desktop", "mobil", "Tablet"]
weights_device = [0.4, 0.4, 0.2]
link = {"ИПОТЕКА": 1, "ОСАГО": 2, "ДОМ": 3, "ДМС": 4, "КАСКО": 50}
weights_link = [0.1, 0.4, 0.1, 0.15, 0.25]
weights_action = [0.6, 0.3, 0.1]

In [ ]:
data_list = []
for i in range(10):
    session_start = datetime.datetime.now()
    session_end = datetime.datetime.now() + datetime.timedelta(milliseconds=random.randint(1000, 1000000))
    name_link = random.choices(list(link.keys()), weights=weights_link)[0] if (session_end - session_start).seconds > 15 else None
    action_type = random.choices(['calculation', 'view', 'add to cart'], weights=weights_action)[0] if name_link is not None else 'view'
    data = {"user_id": int(random.randint(100, 500000) * (time.time() * 1000 % 10)),
            "session_id": str(uuid.uuid4()).split('-')[0],
            "traffic_source": random.choice(['organic', 'paid', 'social', 'email', 'referral', 'direct']),
            "session_start": session_start,
            "session_end": session_end,
            "device": random.choices(device, weights=weights_device)[0],
            "name_link": name_link,
            "action_type": action_type,
            "purchase_amount": round(random.uniform(1000 * (link[name_link] / 2), 10000 * link[name_link]), 2) if (action_type == "calculation" and name_link is not None) else None,
            }
    data_list.append(data)

In [7]:
df = pandas.DataFrame(data_list)
duckdb.sql("from df").show()    

┌─────────┬────────────┬────────────────┬────────────────────────────┬────────────────────────────┬─────────┬───────────┬─────────────┬─────────────────┐
│ user_id │ session_id │ traffic_source │       session_start        │        session_end         │ device  │ name_link │ action_type │ purchase_amount │
│  int64  │  varchar   │    varchar     │        timestamp_ns        │        timestamp_ns        │ varchar │  varchar  │   varchar   │     double      │
├─────────┼────────────┼────────────────┼────────────────────────────┼────────────────────────────┼─────────┼───────────┼─────────────┼─────────────────┤
│ 1054547 │ ed65d75f   │ referral       │ 2025-07-11 20:07:52.436861 │ 2025-07-11 20:08:27.436864 │ mobil   │ КАСКО     │ calculation │       153705.88 │
│ 3209074 │ 6e059f1b   │ email          │ 2025-07-11 20:07:52.436966 │ 2025-07-11 20:10:45.344966 │ desctop │ ИПОТЕКА   │ view        │            NULL │
│ 2830372 │ 844bc05e   │ social         │ 2025-07-11 20:07:52.436994 │ 2025-

In [8]:
conn_str = (
    "host=0.0.0.0 "
    "port=5433 "
    "dbname=analytical_db "
    "user=postgres "
    "password=postgres"
)

In [9]:
duckdb.sql(f""" ATTACH '{conn_str}' AS pg_db (TYPE postgres);
                 """)

In [10]:
duckdb.sql(f""" insert into pg_db.stg_analytical.action_users
                select * from df; """)

In [1]:
from pyspark.sql import SparkSession
import os

In [7]:
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .config("spark.ui.port", "4045") \
    .config("spark.driver.host", "10.130.0.11") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

In [8]:
# Чтение из Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "cdc.stg_analytical.action_users") \
    .option("startingOffsets", "latest") \
    .load()
    

In [ ]:
query = df.writeStream \
    .format("console") \
    .start()

query.awaitTermination()

In [10]:
spark.stop()